# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from markdown_it import MarkdownIt
from markdown_it.tree import SyntaxTreeNode
from rdflib import Graph, URIRef, Literal

In [ ]:
#| export
def extract_prefixes(ast):
    """Extract prefixes and base URI from a Markdown-ld AST."""
    prefixes = {}
    base_uri = None
    
    # Look for level-1 heading followed by a code block
    for i, node in enumerate(ast.children):
        if node.type == 'heading' and node.tag == 'h1':
            # Check if next node is a code block
            if i+1 < len(ast.children) and ast.children[i+1].type == 'code':
                code_block = ast.children[i+1]
                # Parse the code block content for prefixes
                for line in code_block.content.split('\n'):
                    line = line.strip()
                    if line.startswith('@prefix'):
                        # Extract prefix and URI
                        parts = line.split(' ', 2)
                        if len(parts) >= 3:
                            prefix = parts[1].rstrip(':')
                            uri = parts[2].rstrip(' .')
                            if uri.startswith('<') and uri.endswith('>'): 
                                uri = uri[1:-1]
                            prefixes[prefix] = uri
                    elif line.startswith('@base'):
                        # Extract base URI
                        parts = line.split(' ', 1)
                        if len(parts) >= 2:
                            uri = parts[1].rstrip(' .')
                            if uri.startswith('<') and uri.endswith('>'): 
                                uri = uri[1:-1]
                            base_uri = uri
    
    return prefixes, base_uri

In [ ]:
#| export
def extract_triples(ast):
    """Extract RDF triples from a Markdown-ld AST."""
    triples = []
    current_subject = None
    current_predicate = None
    
    # Helper function to extract text from inline code
    def extract_code_value(node):
        for child in node.children:
            if child.type == 'inline':
                for inline_child in child.children:
                    if inline_child.type == 'code_inline':
                        return inline_child.content
        return None
    
    # Process each node in the AST
    for node in ast.children:
        # Handle level-2 headings (subjects)
        if node.type == 'heading' and node.tag == 'h2':
            # The next node should be a paragraph with inline code
            if node.next_sibling and node.next_sibling.type == 'paragraph':
                current_subject = extract_code_value(node.next_sibling)
                current_predicate = None
        
        # Handle level-3 headings (predicates)
        elif node.type == 'heading' and node.tag == 'h3':
            if node.next_sibling and node.next_sibling.type == 'paragraph':
                current_predicate = extract_code_value(node.next_sibling)
        
        # Handle bullet lists (objects)
        elif node.type == 'bullet_list' and current_subject and current_predicate:
            for list_item in node.children:
                if list_item.type == 'list_item':
                    # Extract object from list item
                    obj = None
                    for paragraph in list_item.children:
                        if paragraph.type == 'paragraph':
                            obj = extract_code_value(paragraph)
                    
                    if obj:
                        triples.append((current_subject, current_predicate, obj))
    
    return triples

In [ ]:
#| export
def markdown_ld_to_triples(markdown_text):
    """Convert Markdown-LD text to RDF triples."""
    md = MarkdownIt()
    tokens = md.parse(markdown_text)
    ast = SyntaxTreeNode(tokens)
    triples = extract_triples(ast)
    prefixes, base_uri = extract_prefixes(ast)
    return triples, prefixes, base_uri

In [ ]:
#| export
def triples_to_turtle(triples):
    """Convert triples to grouped Turtle format using RDFLib."""
    g = triples_to_rdflib_graph(triples)
    return g.serialize(format="turtle")

In [ ]:
#| export
def markdown_ld_to_turtle(markdown_text):
    """Convert Markdown-LD text to Turtle format using RDFLib's serializer."""
    triples, prefixes, base_uri = markdown_ld_to_triples(markdown_text)
    g = triples_to_rdflib_graph(triples, prefixes, base_uri)
    return g.serialize(format="turtle")

In [ ]:
#| export
def triples_to_rdflib_graph(triples, prefixes=None, base_uri=None):
    """Convert our triples to an RDFLib graph with prefixes."""
    g = Graph()
    
    # Set prefixes
    if prefixes:
        for prefix, uri in prefixes.items():
            g.bind(prefix, uri)
    else:
        # Default prefixes if none provided
        g.bind("schema", "https://schema.org/")
        g.bind("rdfs", "http://www.w3.org/2000/01/rdf-schema#")
        g.bind("xsd", "http://www.w3.org/2001/XMLSchema#")
    
    # Set base URI if provided
    if base_uri:
        g.base = base_uri
    
    # Process triples
    for subject, predicate, obj in triples:
        # Remove angle brackets if present
        if subject.startswith('<') and subject.endswith('>'): subject = subject[1:-1]
        if predicate.startswith('<') and predicate.endswith('>'): predicate = predicate[1:-1]
        
        # Handle literals vs URIs for objects
        if obj.startswith('"'):
            # Check for datatyped literals
            if '"^^' in obj:
                value_part, datatype_part = obj.split('^^', 1)
                value = value_part.strip('"')
                if datatype_part.startswith('<') and datatype_part.endswith('>'): 
                    datatype_part = datatype_part[1:-1]
                obj_val = Literal(value, datatype=URIRef(datatype_part))
            else:
                obj_val = Literal(obj.strip('"'))
        elif obj.isdigit():
            obj_val = Literal(int(obj))
        else:
            if obj.startswith('<') and obj.endswith('>'): obj = obj[1:-1]
            obj_val = URIRef(obj)
        
        # Add triple to graph
        g.add((URIRef(subject), URIRef(predicate), obj_val))
    
    return g

In [ ]:
#| export
def markdown_ld_to_jsonld(markdown_text):
    """Convert Markdown-LD text to JSON-LD format."""
    triples = markdown_ld_to_triples(markdown_text)
    g = triples_to_rdflib_graph(triples)
    return g.serialize(format="json-ld")

In [ ]:
#| export
def markdown_ld_convert(markdown_text, output_format='turtle'):
    """Convert Markdown-LD text to various RDF formats."""
    triples, prefixes, base_uri = markdown_ld_to_triples(markdown_text)
    g = triples_to_rdflib_graph(triples, prefixes, base_uri)
    
    if output_format.lower() in ['turtle', 'ttl']: return g.serialize(format="turtle")
    elif output_format.lower() in ['jsonld', 'json-ld', 'json']: return g.serialize(format="json-ld")
    else: return g.serialize(format=output_format)

In [ ]:
#| export
def main():
    """Command-line interface for markdown-ld."""
    import argparse
    parser = argparse.ArgumentParser(description='Convert Markdown-LD to RDF formats.')
    parser.add_argument('input', help='Input Markdown-LD file')
    parser.add_argument('-o', '--output', help='Output file (default: stdout)')
    parser.add_argument('-f', '--format', default='turtle', 
                        help='Output format (turtle, json-ld, xml, etc.)')
    args = parser.parse_args()
    
    with open(args.input, 'r') as f: markdown_text = f.read()
    
    result = markdown_ld_convert(markdown_text, args.format)
    
    if args.output:
        with open(args.output, 'w') as f: f.write(result)
    else: print(result)

In [ ]:
# Simple Markdown-ld example for testing
markdown_ld_example = """# Example Entity

`<#example-entity>`

## Property One

`<#property-one>`

### Has Value

`<#has-value>`

- `"Some literal value"`
- Another value `42`
"""

In [ ]:
# Test our parser
md = MarkdownIt()
tokens = md.parse(markdown_ld_example)
ast = SyntaxTreeNode(tokens)
triples = extract_triples(ast)
for subject, predicate, obj in triples:
    print(f"{subject} {predicate} {obj} .")

<#property-one> <#has-value> "Some literal value" .
<#property-one> <#has-value> 42 .


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()